In [13]:
from copy import deepcopy
from src.sim import Scenario
from src.trades import Swap
import cProfile
import pstats

In [2]:
scenario = Scenario("baseline", "weth")
sample = scenario.pricepaths[0]
scenario.prepare_for_run()
scenario.prepare_for_trades(sample)

[INFO][22:55:35][src.sim.scenario]-44298: Fetching sim_market from subgraph.
[INFO][22:55:45][src.sim.scenario]-44298: Using 48800 1Inch quotes from 2023-12-07 21:55:42.199184 to 2023-12-14 21:55:42.199184
[INFO][22:55:45][src.utils.poolgraph]-44298: Found 20 valid cycles of length 3.


In [3]:
llamma = scenario.llamma
i = 0
j = 1
size = llamma.get_max_trade_size(i, j)

swap = Swap(llamma, i, j, None)

with llamma.use_snapshot_context():
    _, out_snapshot, _ = llamma.trade(i, j, size)

before = llamma.get_snapshot()
out_stateless, _ = swap.execute(size, use_snapshot_context=True)
after = llamma.get_snapshot()

assert out_snapshot == out_stateless

for (k_before, v_before), (k_after, v_after) in zip(
    before.__dict__.items(), after.__dict__.items()
):
    assert k_before == k_after
    if hasattr(v_before, "__dict__"):
        assert v_before.__dict__ == v_after.__dict__

In [4]:
spool = scenario.stableswap_pools[0]
i = 0
j = 1
size = spool.get_max_trade_size(i, j)

swap = Swap(spool, i, j, None)

with spool.use_snapshot_context():
    _, out_snapshot, _ = spool.trade(i, j, size)

before = spool.get_snapshot()
out_stateless, _ = swap.execute(size, use_snapshot_context=True)
after = spool.get_snapshot()

assert out_snapshot == out_stateless

for (k_before, v_before), (k_after, v_after) in zip(
    before.__dict__.items(), after.__dict__.items()
):
    assert k_before == k_after
    if hasattr(v_before, "__dict__"):
        assert v_before.__dict__ == v_after.__dict__

In [14]:
pool = deepcopy(scenario.stableswap_pools[0])
i = 0
j = 1
amt_in = pool.get_max_trade_size(i, j)

swap = Swap(pool, i, j, None)

amt_out_stateless, _ = swap.execute(amt_in, use_snapshot_context=True)

swap.amt = amt_in
amt_out_statefull, _ = swap.execute(amt_in, use_snapshot_context=False)

assert amt_out_stateless == amt_out_statefull

In [15]:
with cProfile.Profile() as pr:
    swap.execute(amt_in, use_snapshot_context=True)

stats = pstats.Stats(pr)
stats.sort_stats(pstats.SortKey.CUMULATIVE)
stats.print_stats()
stats.dump_stats("spool.prof")

         12 function calls in 0.000 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 /Users/tncintra/Desktop/root/xenophon/clients/curve/crvUSDrisk/src/trades/trade.py:67(execute)
        1    0.000    0.000    0.000    0.000 /Users/tncintra/Desktop/root/xenophon/clients/curve/crvUSDrisk/venv/lib/python3.11/site-packages/crvusdsim/pool/crvusd/stableswap.py:403(get_dy)
        1    0.000    0.000    0.000    0.000 /Users/tncintra/Desktop/root/xenophon/clients/curve/crvUSDrisk/venv/lib/python3.11/site-packages/crvusdsim/pool/crvusd/stableswap.py:328(get_y)
        1    0.000    0.000    0.000    0.000 /Users/tncintra/Desktop/root/xenophon/clients/curve/crvUSDrisk/venv/lib/python3.11/site-packages/crvusdsim/pool/crvusd/stableswap.py:176(get_D)
        1    0.000    0.000    0.000    0.000 /opt/homebrew/Cellar/python@3.11/3.11.6_1/Frameworks/Python.framework/Versions/3.11/lib/pytho

In [16]:
pool = deepcopy(scenario.llamma)
i = 0
j = 1
amt_in = pool.get_max_trade_size(i, j)

swap = Swap(pool, i, j, None)

amt_out_stateless, _ = swap.execute(amt_in, use_snapshot_context=True)

swap.amt = amt_in
amt_out_statefull, _ = swap.execute(amt_in, use_snapshot_context=False)

assert amt_out_stateless == amt_out_statefull

In [17]:
with cProfile.Profile() as pr:
    swap.execute(amt_in, use_snapshot_context=True)

stats = pstats.Stats(pr)
stats.sort_stats(pstats.SortKey.CUMULATIVE)
stats.print_stats()
stats.dump_stats("llamma.prof")

         127 function calls in 0.000 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 /Users/tncintra/Desktop/root/xenophon/clients/curve/crvUSDrisk/src/trades/trade.py:67(execute)
        1    0.000    0.000    0.000    0.000 /Users/tncintra/Desktop/root/xenophon/clients/curve/crvUSDrisk/venv/lib/python3.11/site-packages/crvusdsim/pool/crvusd/LLAMMA.py:1385(get_dy)
        1    0.000    0.000    0.000    0.000 /Users/tncintra/Desktop/root/xenophon/clients/curve/crvUSDrisk/venv/lib/python3.11/site-packages/crvusdsim/pool/crvusd/LLAMMA.py:1342(_get_dxdy)
        1    0.000    0.000    0.000    0.000 /Users/tncintra/Desktop/root/xenophon/clients/curve/crvUSDrisk/venv/lib/python3.11/site-packages/crvusdsim/pool/crvusd/LLAMMA.py:1132(calc_swap_out)
        5    0.000    0.000    0.000    0.000 /Users/tncintra/Desktop/root/xenophon/clients/curve/crvUSDrisk/venv/lib/python3.11/site-p

In [18]:
pool = list(scenario.markets.values())[0]
i = 0
j = 1
amt_in = pool.get_max_trade_size(i, j)

swap = Swap(pool, i, j, None)

amt_out_stateless, _ = swap.execute(amt_in, use_snapshot_context=True)

swap.amt = amt_in
amt_out_statefull, _ = swap.execute(amt_in, use_snapshot_context=False)

assert amt_out_stateless == amt_out_statefull

In [19]:
with cProfile.Profile() as pr:
    swap.execute(amt_in, use_snapshot_context=True)

stats = pstats.Stats(pr)
stats.sort_stats(pstats.SortKey.CUMULATIVE)
stats.print_stats()
stats.dump_stats("market.prof")

         45 function calls in 0.000 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 /Users/tncintra/Desktop/root/xenophon/clients/curve/crvUSDrisk/src/trades/trade.py:67(execute)
        1    0.000    0.000    0.000    0.000 /Users/tncintra/Desktop/root/xenophon/clients/curve/crvUSDrisk/src/modules/market.py:122(trade)
        1    0.000    0.000    0.000    0.000 /Users/tncintra/Desktop/root/xenophon/clients/curve/crvUSDrisk/src/modules/market.py:152(price_impact)
        1    0.000    0.000    0.000    0.000 /Users/tncintra/Desktop/root/xenophon/clients/curve/crvUSDrisk/venv/lib/python3.11/site-packages/scipy/interpolate/_polyint.py:58(__call__)
        1    0.000    0.000    0.000    0.000 /Users/tncintra/Desktop/root/xenophon/clients/curve/crvUSDrisk/venv/lib/python3.11/site-packages/scipy/interpolate/_interpolate.py:748(_evaluate)
        1    0.000    0.000    0.000    0